# ChEMBL Approved Drugs Snapshot Builder

This notebook fetches FDA-approved small-molecule drugs from the ChEMBL API and creates a pre-processed dataset for the Chemical Space Explorer.

**What this does:**
1. Fetches approved drugs (max_phase=4) from ChEMBL
2. Validates and canonicalizes SMILES
3. Extracts therapeutic labels (ATC, target families)
4. Computes molecular properties
5. Generates Morgan fingerprints
6. Computes UMAP embeddings
7. Performs K-Means clustering
8. Saves to Parquet format

## 1. Setup and Imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from pathlib import Path
import warnings

# Add parent directory to path
import sys
sys.path.insert(0, '..')

from src.utils import validate_smiles, load_config
from src.featurization import compute_properties, compute_fingerprints
from src.embedding import compute_umap_embedding
from src.clustering import perform_clustering

warnings.filterwarnings('ignore')

print("✓ Imports successful")

## 2. Load Configuration

In [ ]:
# Load config from parent directory
config = load_config("../config.yaml")

print("Configuration loaded:")
print(f"  - Fingerprint radius: {config['fingerprint']['radius']}")
print(f"  - Fingerprint bits: {config['fingerprint']['n_bits']}")
print(f"  - UMAP n_neighbors: {config['umap']['n_neighbors']}")
print(f"  - Default clusters: {config['kmeans']['default_n_clusters']}")

## 3. Fetch Data from ChEMBL API

This fetches approved drugs using the ChEMBL REST API.

In [ ]:
def fetch_chembl_approved_drugs(max_results=2000):
    """Fetch approved drugs from ChEMBL API."""
    base_url = "https://www.ebi.ac.uk/chembl/api/data/molecule"
    
    params = {
        'max_phase': 4,  # Approved drugs
        'molecule_type': 'Small molecule',
        'format': 'json',
        'limit': 1000  # API limit per request
    }
    
    molecules = []
    offset = 0
    
    print(f"Fetching approved drugs from ChEMBL (max: {max_results})...")
    
    while len(molecules) < max_results:
        params['offset'] = offset
        
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            if 'molecules' not in data or not data['molecules']:
                print("No more molecules available")
                break
            
            molecules.extend(data['molecules'])
            print(f"  Fetched {len(molecules)} molecules...")
            
            offset += params['limit']
            time.sleep(0.5)  # Be nice to the API
            
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
    
    return molecules[:max_results]

# Fetch the data
molecules = fetch_chembl_approved_drugs(max_results=2000)
print(f"\n✓ Fetched {len(molecules)} molecules from ChEMBL")

## 4. Process and Clean Data

Extract relevant fields and validate SMILES strings.

In [ ]:
def process_chembl_data(molecules):
    """Process ChEMBL molecules into DataFrame."""
    records = []
    
    for mol in molecules:
        # Extract relevant fields
        record = {
            'chembl_id': mol.get('molecule_chembl_id'),
            'smiles': mol.get('molecule_structures', {}).get('canonical_smiles'),
            'name': mol.get('pref_name', mol.get('molecule_chembl_id')),
            'max_phase': mol.get('max_phase'),
        }
        
        # Extract ATC classifications if available
        if 'atc_classifications' in mol and mol['atc_classifications']:
            atc = mol['atc_classifications'][0]
            record['ATC Level 1'] = atc[:1] if atc else None
            record['ATC Level 2'] = atc[:3] if len(atc) >= 3 else None
        
        # Extract target information
        if 'molecule_hierarchy' in mol:
            hierarchy = mol['molecule_hierarchy']
            record['Target Family'] = hierarchy.get('protein_class_desc')
        
        records.append(record)
    
    df = pd.DataFrame(records)
    print(f"Created DataFrame with {len(df)} records")
    
    # Validate and canonicalize SMILES
    print("\nValidating SMILES...")
    valid_mask = []
    canonical_smiles = []
    
    for idx, smiles in enumerate(df['smiles']):
        is_valid, canonical = validate_smiles(smiles)
        valid_mask.append(is_valid)
        canonical_smiles.append(canonical)
    
    # Filter valid molecules
    df = df[valid_mask].copy()
    df['smiles'] = [canonical_smiles[i] for i, v in enumerate(valid_mask) if v]
    
    invalid_count = len(valid_mask) - sum(valid_mask)
    print(f"  Valid: {len(df)} molecules")
    print(f"  Invalid: {invalid_count} molecules")
    
    # Add FDA approved flag
    df['FDA Approved'] = True
    
    # Map ATC Level 1 to therapeutic areas
    if 'ATC Level 1' in df.columns:
        atc_mapping = {
            'A': 'Alimentary',
            'B': 'Blood',
            'C': 'Cardiovascular',
            'D': 'Dermatological',
            'G': 'Genitourinary',
            'H': 'Hormones',
            'J': 'Anti-infectives',
            'L': 'Antineoplastic',
            'M': 'Musculoskeletal',
            'N': 'Nervous system',
            'P': 'Antiparasitic',
            'R': 'Respiratory',
            'S': 'Sensory organs',
            'V': 'Various'
        }
        df['Therapeutic Area'] = df['ATC Level 1'].map(atc_mapping).fillna('Unknown')
    
    return df

# Process the data
df = process_chembl_data(molecules)
print(f"\n✓ Processed {len(df)} valid molecules")
print(f"\nColumns: {list(df.columns)}")

## 5. Preview Data

In [ ]:
# Show first few rows
df.head(10)

In [ ]:
# Show data statistics
print("Dataset Statistics:")
print(f"  Total molecules: {len(df)}")
print(f"\nTherapeutic Area distribution:")
if 'Therapeutic Area' in df.columns:
    print(df['Therapeutic Area'].value_counts())

## 6. Compute Molecular Properties

In [ ]:
print("Computing molecular properties...")
df = compute_properties(df)
print("✓ Properties computed")

# Show property statistics
print("\nProperty Statistics:")
print(df[['Molecular Weight', 'LogP', 'TPSA', 'HBA', 'HBD']].describe())

## 7. Compute Morgan Fingerprints

In [ ]:
print("Computing Morgan fingerprints...")
fingerprints = compute_fingerprints(df, config)
print(f"✓ Fingerprints computed: shape {fingerprints.shape}")
print(f"  Fingerprint density: {fingerprints.mean():.3f}")

## 8. Compute UMAP Embedding

In [ ]:
print("Computing UMAP embedding (this may take a few minutes)...")
embedding = compute_umap_embedding(fingerprints, config)
print(f"✓ UMAP embedding computed: shape {embedding.shape}")

# Add to dataframe
df['umap_x'] = embedding[:, 0]
df['umap_y'] = embedding[:, 1]

print(f"  X range: [{embedding[:, 0].min():.2f}, {embedding[:, 0].max():.2f}]")
print(f"  Y range: [{embedding[:, 1].min():.2f}, {embedding[:, 1].max():.2f}]")

## 9. Perform K-Means Clustering

In [ ]:
print("Performing K-Means clustering...")
labels = perform_clustering(embedding, config)
df['cluster'] = labels
print(f"✓ Clustering complete: {len(np.unique(labels))} clusters")

# Show cluster distribution
print("\nCluster distribution:")
print(df['cluster'].value_counts().sort_index())

## 10. Visualize Chemical Space (Preview)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    df['umap_x'], 
    df['umap_y'], 
    c=df['cluster'], 
    cmap='tab20',
    alpha=0.6,
    s=20
)
plt.colorbar(scatter, label='Cluster')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.title('Chemical Space of ChEMBL Approved Drugs')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✓ Visualization complete")

## 11. Save to Parquet

In [ ]:
# Create output directory
output_path = Path("../data/chembl_approved_drugs.parquet")
output_path.parent.mkdir(exist_ok=True)

# Save to parquet
df.to_parquet(output_path, index=False)

print(f"✓ Dataset saved to {output_path}")
print(f"\nFinal dataset:")
print(f"  Total molecules: {len(df)}")
print(f"  Total features: {len(df.columns)}")
print(f"  File size: {output_path.stat().st_size / 1024 / 1024:.2f} MB")
print(f"\nColumns:")
for col in df.columns:
    print(f"  - {col}")

## 12. Summary

The ChEMBL approved drugs dataset is now ready for use in the Chemical Space Explorer!

**Next steps:**
1. Run the Streamlit app: `streamlit run ../app.py`
2. Select "Built-in ChEMBL Drugs" in the sidebar
3. Explore the pre-computed chemical space

**What's included:**
- Validated SMILES strings
- Therapeutic labels (ATC, target families)
- Molecular properties (LogP, MW, TPSA, etc.)
- Pre-computed UMAP embeddings
- Pre-computed cluster assignments